À partir des datasets `population_communes.csv` et `surface_departements.csv`, créer un nouveau dataset qui contient une ligne par département, avec ces colonnes:

- la somme des "Population municipale" du département
- la somme des "Population totale" du département (pour l'explication de la distinction entre "Population municipale" et "Population totale", voir: https://www.insee.fr/fr/metadonnees/definition/c1270)
- la part (en pourcentage) de la population municipale par rapport à la population totale
- la part (en pourcentage) de la population (municipale) du département au sein de sa région
- la densité de la population (municipale) en nb d'habitants / km2

(Le dataset final devrait ressembler à `result-exo-cc.csv`)


In [2]:
import pandas as pd

In [6]:
df_c = pd.read_csv("population_communes.csv")

df_c.head()

,Code département,Code canton,Code arrondissement,Code région,Nom de la commune,Code commune,Nom de la région,variable,value
0,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population totale,184
1,64,27.0,3,75,Aast,1,Nouvelle-Aquitaine,Population municipale,177
2,55,10.0,2,44,Abainville,1,Grand Est,Population totale,310
3,55,10.0,2,44,Abainville,1,Grand Est,Population municipale,305
4,60,11.0,1,32,Abancourt,1,Hauts-de-France,Population totale,658


In [9]:
df_d = pd.read_csv("surface_departements.csv",
                   encoding = "utf-8")

df_d.head()

,code_insee,nom,surf_km2
0,974,La Réunion,2505.0
1,11,Aude,6343.0
2,43,Haute-Loire,5003.0
3,13,Bouches-du-Rhône,5247.0
4,47,Lot-et-Garonne,5385.0


In [42]:
df_municipale = df_c[df_c['variable'] == 'Population municipale'].copy()
df_totale = df_c[df_c['variable'] == 'Population totale'].copy()

In [53]:
df_c = df_c.groupby(['Code région', 'Code département', 'variable']).sum().reset_index()[
    ['Code région', 'Code département', 'variable', 'value']
]

df_c.head()

,Code région,Code département,variable,value
0,1,971,Population municipale,394110
1,1,971,Population totale,400170
2,2,972,Population municipale,376480
3,2,972,Population totale,382294
4,3,973,Population municipale,269352


In [49]:
df_municipale = df_municipale.groupby(['Code département']).sum().copy()
df_municipale = df_municipale.rename(columns={'value': 'Population municipale'})

df_municipale.head()

df_municipale.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 01 to 974
Data columns (total 5 columns):
Code canton              100 non-null float64
Code arrondissement      100 non-null int64
Code région              100 non-null int64
Code commune             100 non-null int64
Population municipale    100 non-null int64
dtypes: float64(1), int64(4)
memory usage: 4.7+ KB


In [46]:
df_totale = df_totale.groupby(['Code département']).sum().copy()

df_totale = df_totale.rename(columns={'value': 'Population totale'})

df_totale.drop(['Code commune'])
df_totale.head()

KeyError: "['Code commune'] not found in axis"